## Imports

In [1]:
import pandas as pd
import numpy as np
import spacy
import nltk
import re


In [2]:
#read in data
behold_products = pd.read_excel('Behold+product+data+04262021.xlsx',header = 0)
attr_tags = pd.read_csv('usc_additional_tags USC.csv')

In [3]:
attr_tags

,product_id,product_color_id,attribute_name,attribute_value
0,01E5ZXP5H0BTEZT9QD2HRZJ47A,01E5ZXP5JCREDC7WJVMWHK5Q40,materialclothing,linenblend
1,01E5ZXP5H0BTEZT9QD2HRZJ47A,01E5ZXP5JCREDC7WJVMWHK5Q40,materialclothing,cottonblend
2,01E5ZXP5H0BTEZT9QD2HRZJ47A,01E5ZXP5JCREDC7WJVMWHK5Q40,style,modern
3,01E5ZXP5H0BTEZT9QD2HRZJ47A,01E5ZXP5JCREDC7WJVMWHK5Q40,style,businesscasual
4,01E5ZXP5H0BTEZT9QD2HRZJ47A,01E5ZXP5JCREDC7WJVMWHK5Q40,style,classic
...,...,...,...,...
97415,01DT2D3BN9K4SEH71MPB7WD5WA,01DT2D3BNFZBEA1BTZHSGCQN46,fit,relaxed
97416,01DT2D3BN9K4SEH71MPB7WD5WA,01DT2D3BNFZBEA1BTZHSGCQN46,lengthblazerscoatsandjackets,midlengthathips
97417,01DT2D3BN9K4SEH71MPB7WD5WA,01DT2D3BNFZBEA1BTZHSGCQN46,closureblazerscoatsandjackets,buttonedfront
97418,01DT2D3BN9K4SEH71MPB7WD5WA,01DT2D3BNFZBEA1BTZHSGCQN46,occasion,work


In [4]:
#pivot df
grouped_tags = pd.DataFrame(attr_tags.groupby(['product_id','attribute_name'])
                            .attribute_value.apply(lambda x:" ".join(set(x)))).reset_index()

grouped_tags

,product_id,attribute_name,attribute_value
0,01DMBRYVA2P5H24WK0HTK4R0A1,category,bottom
1,01DMBRYVA2P5H24WK0HTK4R0A1,primarycolor,beiges
2,01DMBRYVA2PEPWFTT7RMP5AA1T,category,blazerscoatsjackets
3,01DMBRYVA2Q2ST7MNYR6EEY4TK,category,onepiece
4,01DMBRYVA2S5T9W793F4CY41HE,category,accessory
...,...,...,...
56505,01E607BHRQAJDZ76MJFN7RPRK1,primarycolor,greens
56506,01E607BHRQAJDZ76MJFN7RPRK1,rise,highover9
56507,01E607BHRQAJDZ76MJFN7RPRK1,sizing,regular
56508,01E607BHRQAJDZ76MJFN7RPRK1,style,androgynous modern casual


In [5]:
pivoted_tags = grouped_tags.pivot(index='product_id',columns='attribute_name',values='attribute_value').reset_index()
pivoted_tags.fillna('unknown',inplace=True)

In [6]:
(pivoted_tags != 'unknown').apply(sum).sort_values(ascending=False)[1:20]

attribute_name
category             3969
primarycolor         3925
style                3916
occasion             3914
gender               3913
fit                  2949
sizing               2941
materialclothing     2924
drycleanonly         2776
sleevelength         2047
neckline             1774
additionalcolor      1532
lengthtop            1354
pattern              1346
subcategorytop        952
subcategorybottom     892
subcategoryshoe       669
shoewidth             666
toestyle              661
dtype: int64

In [7]:
#function to tokenize columns, removing stopwords and returning lemmas
nlp = spacy.load("en_core_web_lg", disable=['ner', 'parser', "tok2vec"])
def spacy_tokens(line):
    line = re.findall('(?u)\\b[a-zA-Z][a-zA-Z]+\\b',str(line))
    doc = nlp(" ".join(line))
    return " ".join([token.lemma_.lower() for token in doc if token.is_stop != True])
    
spacy_tokens('Our signature khadi shirt\navailable in black .')

'signature khadi shirt available black'

In [8]:
#%%time 1m08s per column
behold_products['token_description'] = behold_products['description'].apply(spacy_tokens)

In [9]:
#build token columns
df_cols = ['description','brand_name','name']

#runs a little slow
def token_builder(df,df_cols):
    for i in df_cols:
        print('parsing token_'+i)
        df['token_'+i] = df[i].apply(spacy_tokens)
        


In [10]:
%%time
token_builder(behold_products,df_cols)

parsing token_description
parsing token_brand_name
parsing token_name
CPU times: user 1min 14s, sys: 257 ms, total: 1min 14s
Wall time: 1min 14s


In [11]:
#136 instances where name not identical to brandname
behold_products[['name','brand_name']][behold_products['name'] != behold_products['brand_name']]

,name,brand_name
18072,OLD Behold Demi Lune Miel,Behold Demi Lune Miel
18073,OLD Behold Flat Hobo Miel,Behold Flat Hobo Miel
18074,OLD Behold Flat Hobo Scarab,Behold Flat Hobo Scarab
18075,OLD Behold Lune Shopper Black Grain,Behold Lune Shopper Black Grain
18076,OLD Behold Lune Shopper Plantation Grain,Behold Lune Shopper Plantation Grain
...,...,...
54483,Ilana Mary Jane,Chocolate Croco
55180,Travel Ballerina,Off White Woven Sheepskin
55181,Travel Ballerina,Off White Sheepskin
55340,CARMEN SANDAL,Black Sheepskin


In [12]:
behold_products[['product_id','brand','token_description','token_brand_name','token_name']]

,product_id,brand,token_description,token_brand_name,token_name
0,01EX0PN4J9WRNZH5F93YEX6QAF,Two,signature khadi shirt available black white ea...,khadi stripe shirt signature shirt,khadi stripe shirt signature shirt
1,01F0C4SKZV6YXS3265JMC39NXW,Collina Strada,mid length dress ruffles adjustable straps bia...,ruffle market dress loopy pink sistine tomato,ruffle market dress loopy pink sistine tomato
2,01EY4Y1BW8VZW51BWG5VZY82XW,Cariuma,ibi slip raw red knit sneaker women,ibi slip raw red knit sneaker women,ibi slip raw red knit sneaker women
3,01EY50E27A0P5V6KCW01XPDB43,Cariuma,ibi slip black knit sneaker women,ibi slip black knit sneaker women,ibi slip black knit sneaker women
4,01EY6DWHC2W5HPNEGXKEJ4A1CX,Cariuma,nan,catiba pro skate black suede canvas contrast t...,catiba pro skate black suede canvas contrast t...
...,...,...,...,...,...
61350,01EYB5ERGYPFNGM6C9QK7Q9EV0,Nomasei,flat bowvida mule black suede ideal spring sum...,bowvida mule black suede kidskin,bowvida mule black suede kidskin
61351,01EHWTBFP368Q035FW95TRJDAY,Nomasei,flat vida mule tangerine suede comfortable fem...,sandale vida mule tangerine suede kidskin,sandale vida mule tangerine suede kidskin
61352,01EHWTCFTPPSCW10D4XBQZF28H,Nomasei,flat bowvida mule fuschia suede ideal spring s...,bowvida mule fuschia suede kidskin,bowvida mule fuschia suede kidskin
61353,01EYB5B5FH7JESXF82ZEMVXZMS,Nomasei,flat vida mule silver metalized leather comfor...,vida mule silver metalized leather,vida mule silver metalized leather


In [13]:
#identify colors and products
#from hw 3

colors = ['Beige','Black','Blue','Brown','Burgundy','Gold','Gray','Green','Multi','Navy','Neutral','Orange','Pink','Purple','Red','Silver','Teal','White','Yellow','leopard','striped','plaid','floral']

products =  ['shirt','Bottom','One Piece','Shoe','Handbag','Scarf','skirt','hoodie','jacket','coat','pajamas', 'blazer',
            'dress','top','blouse','bag','purse','earring','lingerie','bra','jewelry','pants','jewelry','accessories',
           'jeans','belt','raincoat','outerwear','Two Piece', 'hat','sneaker','heels','pumps','camisole','outerwear','sweater']


def stem_join(list):
    doc = nlp(" ".join(list))
    list = [token.lemma_.lower() for token in doc]
    list = '|'.join(r"\b{}\b".format(x) for x in list) #from stackoverflow
    return list

colors = stem_join(colors)
products = stem_join(products)

In [14]:
#identify products and colors
behold_products['product type'] = behold_products['token_name'].str.findall(products)
#data['colors'] = data['token_descriptions'].apply(lambda x:" ".join(x)).str.findall(colors)

In [15]:
behold_products['token_description'][:100].str.findall(products)

0             [shirt, blazer]
1                     [dress]
2                   [sneaker]
3                   [sneaker]
4                          []
               ...           
95    [blazer, jacket, jeans]
96           [pants, sweater]
97              [pants, belt]
98                         []
99                    [shirt]
Name: token_description, Length: 100, dtype: object

In [16]:
behold_products.iloc[95,:]

product_id                                    01DT2D3BN9K4SEH71MPB7WD5WA
brand                                                              Vince
brand_category                                    women:CLOTHING:JACKETS
name                                              Oversized Twill Blazer
details                \nFully lined\nButton fastening at front\nComp...
created_at                                   2019-11-19 17:59:24.578 UTC
brand_canonical_url    https://pink.modaoperandi.com/vince-fw19/overs...
description            Vince is one brand that can be relied on for c...
brand_description      Vince is one brand that can be relied on for c...
brand_name                                        Oversized Twill Blazer
product_active                                                     False
token_description      vince brand relied classic wardrobe staples pr...
token_brand_name                                  oversized twill blazer
token_name                                        o

In [17]:
behold_products

,product_id,brand,brand_category,name,details,created_at,brand_canonical_url,description,brand_description,brand_name,product_active,token_description,token_brand_name,token_name,product type
0,01EX0PN4J9WRNZH5F93YEX6QAF,Two,Unknown,Khadi Stripe Shirt-our signature shirt,NaN,2021-01-27 01:17:19.305 UTC,https://two-nyc.myshopify.com/products/white-k...,Our signature khadi shirt\navailable in black ...,Our signature khadi shirt\n\navailable in blac...,Khadi Stripe Shirt-our signature shirt,True,signature khadi shirt available black white ea...,khadi stripe shirt signature shirt,khadi stripe shirt signature shirt,"[shirt, shirt]"
1,01F0C4SKZV6YXS3265JMC39NXW,Collina Strada,Unknown,RUFFLE MARKET DRESS LOOPY PINK SISTINE TOMATO,NaN,2021-03-09 18:43:10.457 UTC,https://collina-strada-2.myshopify.com/product...,Mid-length dress with ruffles and adjustable s...,Mid-length dress with ruffles and adjustable s...,RUFFLE MARKET DRESS LOOPY PINK SISTINE TOMATO,True,mid length dress ruffles adjustable straps bia...,ruffle market dress loopy pink sistine tomato,ruffle market dress loopy pink sistine tomato,[dress]
2,01EY4Y1BW8VZW51BWG5VZY82XW,Cariuma,Unknown,IBI Slip On Raw Red Knit Sneaker Women,NaN,2021-02-10 02:58:59.591 UTC,https://cariuma.myshopify.com/products/ibi-sli...,IBI Slip On Raw Red Knit Sneaker Women,IBI Slip On Raw Red Knit Sneaker Women,IBI Slip On Raw Red Knit Sneaker Women,False,ibi slip raw red knit sneaker women,ibi slip raw red knit sneaker women,ibi slip raw red knit sneaker women,[sneaker]
3,01EY50E27A0P5V6KCW01XPDB43,Cariuma,Unknown,IBI Slip On Black Knit Sneaker Women,NaN,2021-02-10 03:40:52.842 UTC,https://cariuma.myshopify.com/products/ibi-sli...,IBI Slip On Black Knit Sneaker Women,IBI Slip On Black Knit Sneaker Women,IBI Slip On Black Knit Sneaker Women,False,ibi slip black knit sneaker women,ibi slip black knit sneaker women,ibi slip black knit sneaker women,[sneaker]
4,01EY6DWHC2W5HPNEGXKEJ4A1CX,Cariuma,Unknown,CATIBA PRO Skate Black Suede and Canvas Contra...,NaN,2021-02-10 16:55:13.024 UTC,https://cariuma.myshopify.com/products/catiba-...,NaN,NaN,CATIBA PRO Skate Black Suede and Canvas Contra...,False,nan,catiba pro skate black suede canvas contrast t...,catiba pro skate black suede canvas contrast t...,[sneaker]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61350,01EYB5ERGYPFNGM6C9QK7Q9EV0,Nomasei,SandalsSales,BowVida Mule in Black Suede Kidskin,"Feminine flat mules.\nSquare shape, V line on ...",2021-02-12 13:04:05.149 UTC,https://www.nomasei.com/en/produit/bowvida-mul...,The flat BowVida mule in black suede is the id...,The flat BowVida mule in black suede is the id...,BowVida Mule in Black Suede Kidskin,False,flat bowvida mule black suede ideal spring sum...,bowvida mule black suede kidskin,bowvida mule black suede kidskin,[]
61351,01EHWTBFP368Q035FW95TRJDAY,Nomasei,Flat sandalsArchives,Sandale Vida Mule In Tangerine Suede Kidskin,"Feminine flat mules.\nSquare shape, V line on ...",2020-09-10 20:12:19.514 UTC,https://www.nomasei.com/en/produit/vida-mule-i...,The flat Vida mule in tangerine suede is the c...,The flat Vida mule in tangerine suede is the c...,Sandale Vida Mule In Tangerine Suede Kidskin,False,flat vida mule tangerine suede comfortable fem...,sandale vida mule tangerine suede kidskin,sandale vida mule tangerine suede kidskin,[]
61352,01EHWTCFTPPSCW10D4XBQZF28H,Nomasei,Flat sandalsArchives,BowVida Mule in Fuschia Suede Kidskin,"Feminine flat mules.\nSquare shape, V line on ...",2020-09-10 20:12:52.43 UTC,https://www.nomasei.com/en/produit/mule-bowvid...,The flat BowVida mule in fuschia suede is the ...,The flat BowVida mule in fuschia suede is the ...,BowVida Mule in Fuschia Suede Kidskin,False,flat bowvida mule fuschia suede ideal spring s...,bowvida mule fuschia suede kidskin,bowvida mule fuschia suede kidskin,[]
61353,01EYB5B5FH7JESXF82ZEMVXZMS,Nomasei,SandalsSales,Vida Mule in Silver Metalized Leather,"Feminine flat mules.\nSquare shape, V line on ...",2021-02-12 13:02:07

## Joining DataFrames

Creating a dataframe to find outfits, this is for part II

In [ ]:
outfit_joined=data.merge(outfit, left_on='product_id', right_on='product_id')
outfit_joined.drop(columns=['brand_y'], inplace=True)
outfit_joined.rename(columns={'brand_x':'brand'}, inplace=True)
outfit_joined

## Finding made in

In [18]:
brand=pd.read_csv('behold_brands USC.csv')
outfit=pd.read_csv('outfit_combinations USC.csv')
data=behold_products.copy()
additional=pd.read_csv('usc_additional_tags USC.csv')

In [19]:
data['brand_description'].replace(np.NaN, 'unknown', inplace=True)

In [72]:
data['made_in']=data['brand_description'].str.findall(r'made in([\w\s]+?)(?=\n)', flags=re.IGNORECASE)

In [98]:
types = []
for i in data['made_in']:
    types.append(type(i))

In [116]:
types=pd.DataFrame(types)
types.rename(columns={0:'class'}, inplace=True)

In [117]:
for index, i in enumerate(types['class']):
    if type(i) == float:
        print(index,i,type(i))

In [21]:
# def spacy_countries(x):
#     nlp = spacy.load("en_core_web_lg")
#     #re.findall(r'made in ([\w\s]+)', x,flags=re.IGNORECASE)
#     doc = nlp(x)
#     return [ent.text for ent in doc.ents if ent.label_=='GPE']

# spacy_countries('made in the USA \n Dry cleaner')


[]

In [71]:
data['made_in'].apply(lambda x: " ".join(set(x)) if len(x)>0 else 'unknown')

TypeError: object of type 'float' has no len()

In [22]:
#data['brand_description'].head(20).apply(lambda x: spacy_countries(str(x)))

In [23]:
# nlp = spacy.load('en_core_web_lg')
# country = []
# for i in data['brand_description']:
#     doc= nlp(i)
#     for ent in doc.ents:
#         doc_countries = []
#         if ent.label_ == 'GPE':
#             doc_countries.append(ent.text)
#         else:
#             doc_countries.append('unknown')
#         country.append(doc_countries)
#         print(doc)

# Finding Attributes

## Sizing and Fit

In [46]:
joined_attributes=data.merge(pivoted_tags, on='product_id', how='left')

In [47]:
joined_attributes.replace({np.NaN:'unknown'}, inplace=True)
joined_attributes['sizing'].replace('unknown','unknown_size', inplace=True)
joined_attributes['fit'].replace('unknown','unknown_fit', inplace=True)

In [48]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()
enc = OneHotEncoder(handle_unknown='ignore')

for i in ['sizing', 'fit']:
    
    #Adding labels
    joined_attributes[i]=labelencoder.fit_transform(joined_attributes[i])

    #OneHotEncoding
    enc_df = pd.DataFrame(enc.fit_transform(joined_attributes[[i]]).toarray())
    
    # merge with main df bridge_df on key values
    joined_attributes = joined_attributes.join(enc_df)

    #renaming columns
    class_encodings = labelencoder.classes_
    for i in range(len(class_encodings)):
        joined_attributes.rename(columns={i:class_encodings[i]}, inplace=True)

In [50]:
joined_attributes.head()

,product_id,brand,brand_category,name,details,created_at,brand_canonical_url,description,brand_description,brand_name,...,oversized semifitted,relaxed,relaxed straightregular,semifitted,semifitted relaxed,semifitted straightregular,semifitted straightregular relaxed,straightregular,straightregular relaxed,unknown_fit
0,01EX0PN4J9WRNZH5F93YEX6QAF,Two,Unknown,Khadi Stripe Shirt-our signature shirt,unknown,2021-01-27 01:17:19.305 UTC,https://two-nyc.myshopify.com/products/white-k...,Our signature khadi shirt\navailable in black ...,Our signature khadi shirt\n\navailable in blac...,Khadi Stripe Shirt-our signature shirt,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,01F0C4SKZV6YXS3265JMC39NXW,Collina Strada,Unknown,RUFFLE MARKET DRESS LOOPY PINK SISTINE TOMATO,unknown,2021-03-09 18:43:10.457 UTC,https://collina-strada-2.myshopify.com/product...,Mid-length dress with ruffles and adjustable s...,Mid-length dress with ruffles and adjustable s...,RUFFLE MARKET DRESS LOOPY PINK SISTINE TOMATO,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,01EY4Y1BW8VZW51BWG5VZY82XW,Cariuma,Unknown,IBI Slip On Raw Red Knit Sneaker Women,unknown,2021-02-10 02:58:59.591 UTC,https://cariuma.myshopify.com/products/ibi-sli...,IBI Slip On Raw Red Knit Sneaker Women,IBI Slip On Raw Red Knit Sneaker Women,IBI Slip On Raw Red Knit Sneaker Women,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,01EY50E27A0P5V6KCW01XPDB43,Cariuma,Unknown,IBI Slip On Black Knit Sneaker Women,unknown,2021-02-10 03:40:52.842 UTC,https://cariuma.myshopify.com/products/ibi-sli...,IBI Slip On Black Knit Sneaker Women,IBI Slip On Black Knit Sneaker Women,IBI Slip On Black Knit Sneaker Women,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,01EY6DWHC2W5HPNEGXKEJ4A1CX,Cariuma,Unknown,CATIBA PRO Skate Black Suede and Canvas Contra...,unknown,2021-02-10 16:55:13.024 UTC,https://cariuma.myshopify.com/products/catiba-...,unknown,unknown,CATIBA PRO Skate Black Suede and Canvas Contra...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [52]:
#not sure what we can do with this one. Already seems to have all categories out and onehot will make too many features
joined_attributes['materialclothing'].unique()

array(['unknown', 'synthetic cottonblend', 'denim cottonblend',
       'purecotton cottonblend', 'cottonblend', 'puresilk',
       'woolblend cashmereblend', 'synthetic viscose',
       'polyester synthetic viscose', 'synthetic', 'linenblend viscose',
       'denim elastane', 'leather', 'ramie',
       'elastane synthetic cottonblend', 'purecotton',
       'polyester cottonblend', 'polyester synthetic',
       'elastane synthetic viscose', 'silkblend cottonblend', 'purelinen',
       'synthetic fauxfur', 'spandex modal synthetic', 'denim purecotton',
       'linenblend rayon', 'rayon viscose', 'viscose rayon',
       'synthetic puresilk', 'purewool', 'viscose cottonblend',
       'elastane cottonblend', 'silkblend synthetic', 'purecashmere',
       'polyester woolblend', 'fauxfur', 'polyester rayon', 'polyester',
       'viscose puresilk', 'silkblend viscose', 'nylon woolblend',
       'mohair woolblend', 'alpaca woolblend', 'silkblend spandex',
       'polyester viscose', 'polyester a

## Creating encodings for dry clean

In [128]:
joined_attributes['drycleanonly'].replace('yes no', 'either', inplace=True)
joined_attributes['drycleanonly'].replace('unknown', np.NaN, inplace=True)
joined_attributes['drycleanonly'].unique()

array([nan, 'no', 'yes', 'either'], dtype=object)

In [59]:
joined_attributes['drycleanonly_encoded']=labelencoder.fit_transform(joined_attributes['drycleanonly'])

#Making class 3 null again because its unknown
joined_attributes['drycleanonly_encoded'].replace(3, np.NaN, inplace=True)

In [61]:
labelencoder.classes

array(['either', 'no', 'yes', nan], dtype=object)

# Final features from attributes

In [126]:
new_features=joined_attributes.iloc[:,102:]
new_features

,maternity,plus regular,regular,regular plus,tall regular,unknown_size,fittedtailored,fittedtailored relaxed,fittedtailored semifitted,oversized,...,relaxed,relaxed straightregular,semifitted,semifitted relaxed,semifitted straightregular,semifitted straightregular relaxed,straightregular,straightregular relaxed,unknown_fit,drycleanonly_encoded
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61350,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN
61351,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN
61352,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN
61353,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN
